In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
#from datetime import datetime
from datetime import timedelta
import pandas as pd
import urllib3
import datetime
import time

# -*- coding: utf -*-

In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def find_between(s, first, last):
    try:
        start = s.index(first) + len(first)
        end = s.index(last, start)
        return s[start:end]
    except ValueError:
        return ""
    
def cut_string(s, cut):
    try:
        cut_from = s.index(cut) + len(cut)
        return s[cut_from:]
    except ValueError:
        return ""

In [52]:
class forecast(object):
    def __init__(max_temp, min_temp, proc_date, acc_date):
        self.max_temp = max_temp
        self.min_temp = min_temp
        self.proc_date = proc_date
        self.acc_date = acc_date
        
data = {}
url = 'https://www.wetter.de/deutschland/wetter-berlin-18228265/wetterprognose.html'
http = urllib3.PoolManager()
soup = BeautifulSoup(http.request('GET',url).data)

daily_periods_dict = {}
current_time = pd.Timestamp(datetime.datetime.now())

proc_date = []
temp = []
rain = []
wind = []
condition = []
rain_l = []

for day in range(15):
    for h in range(4):
        dt = (current_time + timedelta(days=day)).date()
        proc_date.append(datetime.datetime.combine(dt,datetime.time(h*6+2)))
        
period_forcast = soup.findAll("div", {"class":'forecast-column column-1 wt-border-radius-6'})
for period in period_forcast:
    
    temp.append(int(period.find('div', {'class':"forecast-text-temperature wt-font-light"}).text[:-1]))
    condition.append(period.find('div', {'class':"forecast-column-condition"}).text)
    
    rain_html = period.find("div", {"class":'forecast-column-rain'})
    
    r = rain_html.findAll('span', {'class':"wt-font-semibold"})
    if len(r) > 1:
        rain.append(int(r[0].text[:-1]))
        rain_l.append(r[1].text[:-4])
    else:
        rain.append(int(rain_html.find('span', {'class':"wt-font-semibold"}).text[:-1]))    
        rain_l.append(None)
    
    wind_html = period.find("div", {"class":'forecast-column-wind'})
    wind.append(int(wind_html.find('span', {'class':"wt-font-semibold"}).text[1:-5])) 
        
daily_periods_dict['website'] = 'https://www.wetter.de/deutschland/wetter-berlin-18228265/wetterprognose.html'
daily_periods_dict['date_for_which_weather_is_predicted'] = proc_date
daily_periods_dict['city'] = 'Berlin'
daily_periods_dict['date_of_acquisition'] = current_time

daily_periods_dict['temperature'] = temp
daily_periods_dict['wind_speed'] = wind
daily_periods_dict['precipation_per'] = rain

daily_periods_dict['precipation_l'] = rain_l
daily_periods_dict['wind_direction'] = None
daily_periods_dict['condition'] = condition

daily = pd.DataFrame(daily_periods_dict)

/home/liz/anaconda3/envs/bccn/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/liz/anaconda3/envs/bccn/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/liz/anaconda3/envs/bccn/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


      city  condition date_for_which_weather_is_predicted  \
0   Berlin       klar                 2018-05-22 02:00:00   
1   Berlin     sonnig                 2018-05-22 08:00:00   
2   Berlin     sonnig                 2018-05-22 14:00:00   
3   Berlin     sonnig                 2018-05-22 20:00:00   
4   Berlin       klar                 2018-05-23 02:00:00   
5   Berlin     sonnig                 2018-05-23 08:00:00   
6   Berlin     sonnig                 2018-05-23 14:00:00   
7   Berlin     sonnig                 2018-05-23 20:00:00   
8   Berlin       klar                 2018-05-24 02:00:00   
9   Berlin     sonnig                 2018-05-24 08:00:00   
10  Berlin     sonnig                 2018-05-24 14:00:00   
11  Berlin     sonnig                 2018-05-24 20:00:00   
12  Berlin       klar                 2018-05-25 02:00:00   
13  Berlin     sonnig                 2018-05-25 08:00:00   
14  Berlin     sonnig                 2018-05-25 14:00:00   
15  Berlin     sonnig   